In [1]:
import keras

Using TensorFlow backend.


In [2]:
import pandas as pd
dataset=pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
x=dataset.iloc[:,3:13]
y=dataset.iloc[:,13]
#x.head()
#y.head()

In [4]:
gender=pd.get_dummies(x['Gender'],drop_first='True')
geography=pd.get_dummies(x['Geography'],drop_first='True')

In [5]:
x=pd.concat([x,gender,geography],axis=1)
x.drop(['Gender','Geography'],axis=1,inplace=True)

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [7]:
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
x_train=StandardScaler().fit_transform(x_train)
x_test=StandardScaler().fit_transform(x_test)

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, BatchNormalization
from keras.layers import Dropout
from keras.layers import LeakyReLU, PReLU,ELU
from keras.activations import sigmoid
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [15]:
def create_model(layers,activation):
    model=Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(units=nodes,input_dim=x_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(units=nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
    model.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [16]:
model=KerasClassifier(build_fn=create_model,verbose=0)

In [17]:
layers=[[20],[40,20],[45,30,15]]
activation=['relu','sigmoid']

In [18]:
para_grid=dict(layers=layers,activation=activation,batch_size=[128,256],epochs=[30])
grid=GridSearchCV(estimator=model,param_grid=para_grid,cv=5)

In [19]:
grid_result=grid.fit(x_train,y_train)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [20]:
grid_result.best_score_
grid_result.best_params_

{'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': [40, 20]}

In [21]:
pred=grid.predict(x_test)
pred=(pred>0.5)

In [24]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm=confusion_matrix(pred,y_test)
score=accuracy_score(pred,y_test)
print('confusion_matrix:\n',cm)
print('Accuracy Score:',score)

confusion_matrix:
 [[1562  231]
 [  33  174]]
Accuracy Score: 0.868


In [ ]:
#classifier=Sequential()
#classifier.add(Dense(output_dim=6, init='he_uniform',activation='relu',input_dim=11))
#classifier.add(Dense(output_dim=6, init='he_uniform',activation='relu'))
#classifier.add(Dense(output_dim=1, init='glorot_uniform',activation='sigmoid'))

In [ ]:
#classifier.compile(optimizer='Adamax',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
#classifier.summary()

In [ ]:
#model_history=classifier.fit(x_train,y_train,validation_split=0.33,batch_size=10,nb_epoch=100)

In [ ]:
#y_pred=classifier.predict(x_test)

In [ ]:
#y_pred=(y_pred>0.5)

In [10]:
#from sklearn.metrics import confusion_matrix,accuracy_score
#cm=confusion_matrix(y_test,y_pred)
#accuracy=accuracy_score(y_test,y_pred)